In [45]:
import os
import json
import requests
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [49]:
#Setup env variables
os.environ['SMTP_SERVER']="localhost"
os.environ['SMTP_PORT']="1025"
os.environ['SMTP_USER']="fake-admin"
os.environ['SMTP_PASSWORD']=""
os.environ['SENDER_EMAIL']="domino-deactivator@test.com"


In [17]:
IDLE_DURATION_THRESHOLD_IN_DAYS=5
MAX_EMAILS_BEFORE_USER_DELETED=2
DOMINO_EXTENSIONS_API_SVC="domino-extensions-api-svc"
EMAIL_TRACKER_PATH="/domino/datasets/local/EmailRemindersUserDeactivation/email_tracker.json"




In [58]:
#FIND ALL INACTIVE USERS (1)
def get_token():
    # Fetch the mounted service account token
    uri = os.environ['DOMINO_API_PROXY']
    return requests.get(f"{uri}/access-token").text
def get_inactive_users(no_of_days,include_svc_accounts=False):
    dns_name = f"{DOMINO_EXTENSIONS_API_SVC}.domino-field.svc.cluster.local"
    endpoint = f"user-management/inactivefor/{no_of_days}"
    url = f"http://{dns_name}/{endpoint}"
    payload = {"users":['user-ds-1','user-ds-2']}
    token = get_token()
    my_headers = {"Authorization": f"Bearer {token}",'Content-Type': 'application/json'}
    domino_api_host = os.environ['DOMINO_API_HOST']
    r = requests.get(url,headers=my_headers)
    lst = r.json()['inactive_accounts']
    new_lst=[]
    for l in lst:
        if not l['is_svc_account']:
            new_lst.append(l)
    return new_lst

def deactivate_users(users):
    service_name="domino-extensions-api-svc"
    dns_name = f"{DOMINO_EXTENSIONS_API_SVC}.domino-field.svc.cluster.local"
    endpoint = "user-management/deactivate"
    url = f"http://{dns_name}/{endpoint}"
    payload = {"users":users}
    token = get_token()
    my_headers = {"Authorization": f"Bearer {token}",'Content-Type': 'application/json'}
    r = requests.post(url,headers=my_headers,json=payload)
    print(r)
    return r.json()
    
def get_email_tracker():
    # Define the path to the JSON file
    json_file_path = EMAIL_TRACKER_PATH

    # Check if the file exists
    if not os.path.exists(json_file_path):
        # If the file does not exist, create an empty JSON file
        with open(json_file_path, 'w') as file:
            json.dump({}, file)  # Create an empty JSON object (empty dictionary)
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    return data

def get_emails_by_user():
    domino_api_host = os.environ['DOMINO_API_HOST']
    url = f"{domino_api_host}/api/users/v1/users"
    token = get_token()
    my_headers = {"Authorization": f"Bearer {token}",'Content-Type': 'application/json'}
    out = requests.get(url,headers=my_headers)
    users = out.json()['users']
    email_by_user_name={}
    for u in users:
        email_by_user_name[u['userName']]=u['email']
    return email_by_user_name
# SEND EMAIL TO USERS TO WHOM LESS THAN MAX_EMAILS_BEFORE_USER_DELETED HAVE BEEN SENT 
def send_email(subject, body, recipient):
    # SMTP configuration from environment variables
    smtp_server = os.getenv("SMTP_SERVER")
    smtp_port = int(os.getenv("SMTP_PORT", 587))
    smtp_user = os.getenv("SMTP_USER")
    smtp_password = os.getenv("SMTP_PASSWORD")
    sender_email = os.getenv("SENDER_EMAIL")

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Send email
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        if smtp_password:
            server.starttls()  # Enable TLS
            server.login(smtp_user, smtp_password)
        server.sendmail(sender_email, recipient, msg.as_string())
        server.quit()
        print(f"Email sent to {recipient}")
        return True
    except Exception as e:
        print(f"Failed to send email: {str(e)}")
        return False

In [60]:
#GET INACTIVE USERS TO WHOM EMAILS HAVE BEEN SENT OUT
email_status_by_user = get_email_tracker()

#Get Inactive Users
inactive_user_list = get_inactive_users(IDLE_DURATION_THRESHOLD_IN_DAYS)
print(inactive_user_list)

#DELETE USERS FROM THIS LIST WHAT ARE NOT IN THE INACTIVE LIST JUST FETCHED (1)
users = email_status_by_user.keys()
current_inactive_user_names = [user['user_name'] for user in inactive_user_list]
print(current_inactive_user_names)

update_email_status_by_user={}
for user in current_inactive_user_names:
    if user in email_status_by_user.keys():
        update_email_status_by_user[user]=email_status_by_user[user]
    else:
        update_email_status_by_user[user]=0
print(update_email_status_by_user)

#IDENTIFY USERS TO WHOM EMAIL NEEDS TO BE SENT OUT
email_reminder_candiates={}
for user, email_reminder_count in update_email_status_by_user.items():
    if email_reminder_count<MAX_EMAILS_BEFORE_USER_DELETED:
        email_reminder_candiates[user]=email_reminder_count
        
#IDENTIFY USERS TO BE DEACTIVATED
users_to_deactivate=[]
for user, email_reminder_count in update_email_status_by_user.items():
    if email_reminder_count>=MAX_EMAILS_BEFORE_USER_DELETED:
        users_to_deactivate.append(user)

print(email_reminder_candiates)
print(users_to_deactivate)

emails_by_user = get_emails_by_user()

new_user_counts = {}
for user,count in email_reminder_candiates.items():
    new_count = count + 1
    body = f"You have not logged in for {IDLE_DURATION_THRESHOLD_IN_DAYS} days. Your account will be deactivated"
    result = send_email(f"Email Deactivation Reminder {new_count}",body,emails_by_user[user])
    email_reminder_candiates[user]=new_count

#Update the file
json_file_path = EMAIL_TRACKER_PATH
with open(json_file_path, 'w') as file:
    json.dump(email_reminder_candiates,file) 
print(email_reminder_candiates)

deactivate_users(users_to_deactivate)

[{'is_svc_account': False, 'user_name': 'user-ds-1'}, {'is_svc_account': False, 'user_name': 'user-ds-2'}, {'is_svc_account': False, 'user_name': 'user-ds-3'}]
['user-ds-1', 'user-ds-2', 'user-ds-3']
{'user-ds-1': 2, 'user-ds-2': 2, 'user-ds-3': 2}
{}
['user-ds-1', 'user-ds-2', 'user-ds-3']
{}
<Response [200]>


{'updated_users': ['user-ds-1', 'user-ds-2', 'user-ds-3']}